In [1]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [3]:
ground_truth[:5]

[{'question': 'When does the course begin?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'How can I get the course schedule?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'What is the link for course registration?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'How can I receive course announcements?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'},
 {'question': 'Where do I join the Slack channel?',
  'course': 'data-engineering-zoomcamp',
  'document': 'c02e79ef'}]

In [4]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

c:\Users\khant\Documents\personal\LLM-Zoomcamp-Study\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [6]:
def minsearch_search(query, course):
    boost = {'question': 1.5, 'section': 0.1}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 4627/4627 [00:17<00:00, 257.50it/s]


In [8]:
hit_rate(relevance_total), mrr(relevance_total)

(0.848714069591528, 0.7288235717887772)

In [9]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

In [10]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [11]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [12]:
def minsearch_vector_search(query, course):
    query_vector = pipeline.transform([query])
    results = vindex.search(query_vector, filter_dict={'course': course}, num_results=5)
    return results

In [43]:
minsearch_vector_search(documents[0]['question'], documents[0]['course'])

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'quest

In [13]:
evaluate(ground_truth, lambda q: minsearch_vector_search(q['question'], q['course']))

100%|██████████| 4627/4627 [00:11<00:00, 405.18it/s]


{'hit_rate': 0.48173762697212014, 'mrr': 0.3572833369353793}

In [14]:
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

In [15]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [16]:
evaluate(ground_truth, lambda q: minsearch_vector_search(q['question'], q['course']))

100%|██████████| 4627/4627 [00:15<00:00, 307.63it/s]


{'hit_rate': 0.8210503566025502, 'mrr': 0.6717347453353508}

In [17]:
from qdrant_client import QdrantClient, models

In [18]:
client = QdrantClient("http://localhost:6333")

In [40]:
EMBEDDING_DIMENSIONALITY = 512

collection_name = "zoomcamp-evaluation"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [41]:
points = []
id = 0
model_handle = "jinaai/jina-embeddings-v2-small-en"

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['question'] + " " + doc['text'], model=model_handle),
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course'],
            "question": doc['question'],
            "id": doc['id']
        }
    )
    points.append(point)
    id += 1

In [42]:
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [55]:
def qdrant_search(query, course):
    results = []
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=query,
            model=model_handle 
        ),
        limit=5, # top closest matches
        with_payload=True, #to get metadata in the results
        query_filter=models.Filter(
            must=[
                {
                    "key": "course",
                    "match": {
                        "value": course
                    }
                }
            ]
        )
    )
    results = [result.payload for result in results.points]
    return results

In [56]:
qdrant_search(documents[0]['question'], documents[0]['course'])

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'course': 'data-engineering-zoomcamp',
  'question': 'Course - When will the course start?',
  'id': 'c02e79ef'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'cours

In [57]:
evaluate(ground_truth, lambda q: qdrant_search(q['question'], q['course']))

100%|██████████| 4627/4627 [02:55<00:00, 26.33it/s]


{'hit_rate': 0.9299762264966501, 'mrr': 0.8517722066133576}

In [58]:
import numpy as np

def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

In [59]:
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)

In [60]:
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

In [61]:
pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

,steps,"[('tfidfvectorizer', ...), ('truncatedsvd', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [74]:
cosine_similarities = []
for i, row in df_results.iterrows():
    v_llm = pipeline.transform([row['answer_llm']])
    v_orig = pipeline.transform([row['answer_orig']])
    cosine_similarity = cosine(v_llm[0], v_orig[0])
    cosine_similarities.append(cosine_similarity)

In [76]:
np.mean(cosine_similarities)

np.float64(0.8415841233490402)

In [77]:
from rouge import Rouge
rouge_scorer = Rouge()

rouge_scores = []
for i, row in df_results.iterrows():
    scores = rouge_scorer.get_scores(row.answer_llm, row.answer_orig)[0]
    rouge_scores.append(scores)

In [79]:
np.mean([score['rouge-1']['f'] for score in rouge_scores])

np.float64(0.3516946452113943)